In [1]:
from tvtk.api import tvtk
 
def read_data():# 读入数据
    plot3d = tvtk.MultiBlockPLOT3DReader(
            xyz_file_name="tvtk/plot3d_data/combxyz.bin",#网格文件
            q_file_name="tvtk/plot3d_data/combq.bin",#空气动力学结果文件
            scalar_function_number=100,#设置标量数据数量
            vector_function_number=200#设置矢量数据数量
            )
    plot3d.update()
    return plot3d
 
plot3d = read_data()
grid = plot3d.output.get_block(0)

In [2]:
print(type(plot3d.output))

<class 'tvtk.tvtk_classes.multi_block_data_set.MultiBlockDataSet'>


In [3]:
print(type(plot3d.output.get_block(0)))

<class 'tvtk.tvtk_classes.structured_grid.StructuredGrid'>


In [4]:
print(grid.dimensions)

[57 33 25]


In [5]:
print(grid.points.to_array())

[[ 2.667     -3.77476   23.83292  ]
 [ 2.943465  -3.7482529 23.66556  ]
 [ 3.2198582 -3.7217531 23.49824  ]
 ...
 [15.84669    5.662141  35.74938  ]
 [16.178299   5.662141  35.74938  ]
 [16.51       5.662141  35.74938  ]]


In [6]:
print(grid.point_data.number_of_arrays)

4


In [7]:
print(grid.point_data.scalars.name)

Density


In [8]:
print(grid.point_data.vectors.name)

Velocity


## 标量可视化

In [2]:
from tvtk.api import tvtk
from tvtkfunc import ivtk_scene, event_loop
 
plot3d = tvtk.MultiBlockPLOT3DReader(
        xyz_file_name="tvtk/plot3d_data/combxyz.bin",
        q_file_name="tvtk/plot3d_data/combq.bin",
        scalar_function_number=100, vector_function_number=200
    )#读入Plot3D数据
plot3d.update()#让plot3D计算其输出数据
grid = plot3d.output.get_block(0)#获取读入的数据集对象
 
con = tvtk.ContourFilter()#创建等值面对象  
con.set_input_data(grid)
con.generate_values(3, grid.point_data.scalars.range)#指定轮廓数和数据范围
 
#设定映射器的变量范围属性
m = tvtk.PolyDataMapper(scalar_range = grid.point_data.scalars.range,
                        input_connection=con.output_port)
a = tvtk.Actor(mapper = m)
a.property.opacity = 0.5#设定透明度为0.5
#窗口绘制
win = ivtk_scene(a)
win.scene.isometric_view()
event_loop()

## 矢量可视化
1. tvtk.Glyph3D() 符号化技术
2. tvtk.MaskPoints () 降采样



In [2]:
from tvtk.api import tvtk
from tvtkfunc import ivtk_scene, event_loop
 
#读入PLot3D数据
plot3d = tvtk.MultiBlockPLOT3DReader(
        xyz_file_name="tvtk/plot3d_data/combxyz.bin",
        q_file_name="tvtk/plot3d_data/combq.bin",
        scalar_function_number=100, vector_function_number=200
    )
plot3d.update()
grid = plot3d.output.get_block(0)
 
#对数据集中的数据进行随机选取，每50个点选择一个点
mask = tvtk.MaskPoints(random_mode=True, on_ratio=50)
mask.set_input_data(grid)
#创建表示箭头的PolyData数据集
# glyph_source = tvtk.ConeSource()
glyph_source = tvtk.ArrowSource()
#在Mask采样后的PolyData数据集每个点上放置一个箭头
#箭头的方向、长度和颜色由于点对应的矢量和标量数据决定
glyph = tvtk.Glyph3D(input_connection=mask.output_port,
                      scale_factor=2)
glyph.set_source_connection(glyph_source.output_port)
m = tvtk.PolyDataMapper(scalar_range=grid.point_data.scalars.range,
                        input_connection=glyph.output_port)
a = tvtk.Actor(mapper=m)
 
#窗口绘制
win = ivtk_scene(a)
win.scene.isometric_view()
event_loop()

In [3]:
print(grid.number_of_points)

47025


In [4]:
mask.update()

In [5]:
print(mask.output.number_of_points)

932


## 空间轮廓线可视化


In [ ]:
from tvtk.api import tvtk
from tvtk.common import configure_input
from tvtkfunc import ivtk_scene, event_loop
 
plot3d = tvtk.MultiBlockPLOT3DReader(
        xyz_file_name="tvtk/plot3d_data/combxyz.bin",
        q_file_name="tvtk/plot3d_data/combq.bin",
        scalar_function_number=100, vector_function_number=200
    )#读入Plot3D数据
plot3d.update()#让plot3D计算其输出数据
grid = plot3d.output.get_block(0)#获取读入的数据集对象
 
outline = tvtk.StructuredGridOutlineFilter()#计算表示外边框的PolyData对象
configure_input(outline, grid)#调用tvtk.common.configure_input()
m = tvtk.PolyDataMapper(input_connection=outline.output_port)
a = tvtk.Actor(mapper=m)
a.property.color = 0.3, 0.3, 0.3
 
#窗口绘制
win = ivtk_scene(a)
win.scene.isometric_view()
event_loop()